In [ ]:
#Object-Oriented Design

In [1]:
import pandas as pd

class Product:
    def __init__(self, product_id, name, price, quantity):
        self.product_id = product_id
        self.name = name
        self.price = price
        self.quantity = quantity

    def update_quantity(self, quantity):
        self.quantity += quantity

class Inventory:
    def __init__(self):
        self.products = {}

    def add_product(self, product):
        self.products[product.product_id] = product

    def update_product(self, product_id, quantity):
        if product_id in self.products:
            self.products[product_id].update_quantity(quantity)

    def delete_product(self, product_id):
        if product_id in self.products:
            del self.products[product_id]

    def search_product(self, product_id):
        return self.products.get(product_id, None)

class Sales:
    def __init__(self):
        self.sales_data = []

    def record_sale(self, product, quantity):
        total_cost = product.price * quantity
        sale_record = {
            'Product ID': product.product_id,
            'Name': product.name,
            'Quantity Sold': quantity,
            'Price': product.price,
            'Total Cost': total_cost
        }
        self.sales_data.append(sale_record)

class Store:
    def __init__(self):
        self.inventory = Inventory()
        self.sales = Sales()

    def add_product(self, product):
        self.inventory.add_product(product)

    def process_sale(self, product_id, quantity):
        product = self.inventory.search_product(product_id)
        if product and product.quantity >= quantity:
            product.update_quantity(-quantity)
            self.sales.record_sale(product, quantity)
            return True
        return False

In [ ]:
#File Handling

In [3]:
import csv
import os

def backup_inventory(func):
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        # Backup inventory to a CSV file
        with open('inventory_backup.csv', mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['Product ID', 'Name', 'Price', 'Quantity'])
            for product in args[0].products.values():
                writer.writerow([product.product_id, product.name, product.price, product.quantity])
        return result
    return wrapper

class Inventory:
    # Existing methods...

    @backup_inventory
    def add_product(self, product):
        self.products[product.product_id] = product

    def load_products_from_csv(self, filename='products.csv'):
        with open(filename, mode='r') as file:
            reader = csv.DictReader(file)
            for row in reader:
                product = Product(row['Product ID'], row['Name'], float(row['Price']), int(row['Quantity']))
                self.add_product(product)

    def save_products_to_csv(self, filename='products.csv'):
        with open(filename, mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['Product ID', 'Name', 'Price', 'Quantity'])
            for product in self.products.values():
                writer.writerow([product.product_id, product.name, product.price, product.quantity])

In [ ]:
# Sales Processing

In [4]:
class Sales:
    # Existing methods...

    def save_sales_to_csv(self, filename='sales.csv'):
        with open(filename, mode='w', newline='') as file:
            writer = csv.DictWriter(file, fieldnames=['Product ID', 'Name', 'Quantity Sold', 'Price', 'Total Cost'])
            writer.writeheader()
            writer.writerows(self.sales_data)

    def load_sales_from_csv(self, filename='sales.csv'):
        with open(filename, mode='r') as file:
            reader = csv.DictReader(file)
            for row in reader:
                self.sales_data.append(row)

In [ ]:
#Data Analysis Using Pandas

In [5]:
def analyze_sales_data(products_file='products.csv', sales_file='sales.csv'):
    products_df = pd.read_csv(products_file)
    sales_df = pd.read_csv(sales_file)

    # Total sales for each product
    total_sales = sales_df.groupby('Product ID')['Total Cost'].sum()

    # Daily sales trends
    sales_df['Date'] = pd.to_datetime(sales_df['Date'])
    daily_sales = sales_df.groupby(sales_df['Date'].dt.date)['Total Cost'].sum()

    # Find the product with the highest sales revenue
    highest_sales_product = sales_df.groupby('Product ID')['Total Cost'].sum().idxmax()

    return total_sales, daily_sales, highest_sales_product

In [ ]:
#NumPy-Based Calculations

In [6]:
import numpy as np

def calculate_sales_statistics(sales_df):
    total_revenue = np.sum(sales_df['Total Cost'])
    average_sales_per_day = np.mean(sales_df.groupby(sales_df['Date'].dt.date)['Total Cost'].sum())
    total_stock_value = np.sum(sales_df['Quantity Sold'] * sales_df['Price'])

    return total_revenue, average_sales_per_day, total_stock_value

In [ ]:
#Testing and Decorators

In [7]:
import unittest

class TestStoreInventory(unittest.TestCase):
    def setUp(self):
        self.store = Store()
        self.product1 = Product('001', 'Widget', 10.0, 100)
        self.store.add_product(self.product1)

    def test_add_product(self):
        self.assertEqual(len(self.store.inventory.products), 1)

    def test_process_sale(self):
        self.store.process_sale('001', 5)
        self.assertEqual(self.product1.quantity, 95)

if __name__ == '__main__':
    unittest.main()

E
ERROR: /root/ (unittest.loader._FailedTest)
----------------------------------------------------------------------
AttributeError: module '__main__' has no attribute '/root/'

----------------------------------------------------------------------
Ran 1 test in 0.003s

FAILED (errors=1)


SystemExit: True

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
